# Assignment 4 - Questions
**Use the merged data frame from Assignment 1,2 and 3 for this assignment**

This Assignment focuses on data preprocessing and model building. Continue with the datasets loaded in Assignment 1, 2 and 3 (or reload with same steps and create merged data frame). In this assignment you need to use Neural Network.

In [1]:
#import required libraries like pandas, numpy, matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import random
from numpy import mean
from numpy import std
from scipy import stats
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_absolute_error

1.Load the data (both features and label data set as before)

In [2]:
#read the train data into dataframe
dengue_train = pd.read_csv("dengue_features_train.csv") 
#view first 3 rows of the dataframe
dengue_train.head(3)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.12260,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.16990,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.03225,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4


In [3]:
#read the dengue_labels_train dataset into another dataframe
dengue_labels = pd.read_csv("dengue_labels_train.csv")  
dengue_labels.head(3)

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4


In [4]:
#merge the above two dataframes on city, year and weekofyear and store it into new dataframe
merged_train_labels = pd.merge(dengue_train,dengue_labels, on= ["city",'year','weekofyear'])
print(merged_train_labels.shape)

(1456, 25)


In previous assignment, we have removed outliers to perform further analysis and so we will be using dataframe without outliers for this model as well.

In [5]:
#calculating the z-score of total_cases for city iq
city_iq = merged_train_labels[merged_train_labels["city"] == "iq"]
print(city_iq["total_cases"].count())
print(city_iq["total_cases"].mean())
print(city_iq["total_cases"].std())
z_iq = np.abs(stats.zscore(city_iq["total_cases"])) 
threshold = 3
print(np.where(z_iq > 3))
#removing the outliers from city iq
oulier_rem_city_iq = city_iq[(z_iq < 3)]
print(oulier_rem_city_iq.shape)

520
7.565384615384615
10.765478342208679
(array([230, 231, 391, 429, 431, 432, 433], dtype=int64),)
(513, 25)


In [6]:
#calculating the z-score of total_cases for city sj
city_sj = merged_train_labels[merged_train_labels["city"] == "sj"]
print(city_sj["total_cases"].count())
print(city_sj["total_cases"].mean())
print(city_sj["total_cases"].std())
z_sj = np.abs(stats.zscore(city_sj["total_cases"]))
threshold = 3
print(np.where(z_sj > 3))
#removing the outliers from city sj
oulier_rem_city_sj = city_sj[(z_sj < 3)]
print(oulier_rem_city_sj.shape)

936
34.18055555555556
51.38137249715158
(array([227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 428, 429, 430, 431, 432, 433], dtype=int64),)
(916, 25)


In [7]:
#creating a new dataframe without outliers
df_without_outliers = pd.concat([oulier_rem_city_iq,oulier_rem_city_sj] )
print(df_without_outliers.shape)
df_without_outliers

(1429, 25)


,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
936,iq,2000,26,2000-07-01,0.192886,0.132257,0.340886,0.247200,25.41,296.740000,...,92.418571,25.41,16.651429,8.928571,26.400000,10.775000,32.5,20.7,3.0,0
937,iq,2000,27,2000-07-08,0.216833,0.276100,0.289457,0.241657,60.61,296.634286,...,93.581429,60.61,16.862857,10.314286,26.900000,11.566667,34.0,20.8,55.6,0
938,iq,2000,28,2000-07-15,0.176757,0.173129,0.204114,0.128014,55.52,296.415714,...,95.848571,55.52,17.120000,7.385714,26.800000,11.466667,33.0,20.7,38.1,0
939,iq,2000,29,2000-07-22,0.227729,0.145429,0.254200,0.200314,5.60,295.357143,...,87.234286,5.60,14.431429,9.114286,25.766667,10.533333,31.5,14.7,30.0,0
940,iq,2000,30,2000-07-29,0.328643,0.322129,0.254371,0.361043,62.76,296.432857,...,88.161429,62.76,15.444286,9.500000,26.600000,11.480000,33.3,19.1,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,sj,2008,13,2008-03-25,0.077850,-0.039900,0.310471,0.296243,27.19,296.958571,...,74.247143,27.19,13.644286,2.885714,25.042857,5.785714,30.0,21.1,1.8,4
932,sj,2008,14,2008-04-01,-0.038000,-0.016833,0.119371,0.066386,3.82,298.081429,...,74.600000,3.82,14.662857,2.714286,26.242857,6.814286,30.6,22.2,0.5,3
933,sj,2008,15,2008-04-08,-0.155200,-0.052750,0.137757,0.141214,16.96,297.460000,...,75.027143,16.96,14.184286,2.185714,25.000000,5.714286,29.4,21.7,30.7,1
934,sj,2008,16,2008-04-15,0.001800,NaN,0.203900,0.209843,0.00,297.630000,...,72.285714,0.00,13.858571,2.785714,25.314286,6.242857,29.4,21.7,11.2,3


In [8]:
#create a copy of the original merged dataframe
dengue = df_without_outliers.copy()

2.Preprocess the data - briefly comment if any special preprocessing is adopted to suit Neural Network

In [9]:
#dataset from previous assignment
current_dataset = dengue.drop(["precipitation_amt_mm","reanalysis_sat_precip_amt_mm","reanalysis_dew_point_temp_k"
                                                    ,"reanalysis_specific_humidity_g_per_kg","reanalysis_precip_amt_kg_per_m2","reanalysis_avg_temp_k",
                                                    "station_precip_mm","station_min_temp_c","station_max_temp_c","station_avg_temp_c","year","weekofyear"],axis=1)
current_dataset.head()

,city,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,reanalysis_air_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_relative_humidity_percent,reanalysis_tdtr_k,station_diur_temp_rng_c,total_cases
936,iq,2000-07-01,0.192886,0.132257,0.340886,0.247200,296.740000,307.3,293.1,92.418571,8.928571,10.775000,0
937,iq,2000-07-08,0.216833,0.276100,0.289457,0.241657,296.634286,306.6,291.1,93.581429,10.314286,11.566667,0
938,iq,2000-07-15,0.176757,0.173129,0.204114,0.128014,296.415714,304.5,292.6,95.848571,7.385714,11.466667,0
939,iq,2000-07-22,0.227729,0.145429,0.254200,0.200314,295.357143,303.6,288.6,87.234286,9.114286,10.533333,0
940,iq,2000-07-29,0.328643,0.322129,0.254371,0.361043,296.432857,307.0,291.5,88.161429,9.500000,11.480000,0


In [10]:
#converting the reanalysis_relative_humidity_percent value to decimal
current_dataset["reanalysis_relative_humidity_percent"] = current_dataset["reanalysis_relative_humidity_percent"]/100

In [11]:
current_dataset['ndvi_ne'] = np.where(current_dataset["ndvi_ne"] <= 0.25, 0,1) 
current_dataset['ndvi_nw'] = np.where(current_dataset["ndvi_ne"] <= 0.25, 0,1) 
current_dataset['ndvi_se'] = np.where(current_dataset["ndvi_se"] <= 0.25, 0,1) 
current_dataset['ndvi_sw'] = np.where(current_dataset["ndvi_sw"] <= 0.25, 0,1) 

In [12]:
#create new columns for seasonal features by extracting the month from week_start_date
current_dataset["month"] = pd.to_datetime(current_dataset["week_start_date"]).dt.month
current_dataset["Jan_Apr"]= np.where(current_dataset["month"].isin([1,2,3,4]),1,0)
current_dataset["May_Aug"]= np.where(current_dataset["month"].isin([5,6,7,8]),1,0)
current_dataset["Sep_Dec"]= np.where(current_dataset["month"].isin([9,10,11,12]),1,0)
current_dataset = current_dataset.drop(["month","week_start_date"],axis =1)

In [13]:
current_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1429 entries, 936 to 935
Data columns (total 15 columns):
city                                    1429 non-null object
ndvi_ne                                 1429 non-null int32
ndvi_nw                                 1429 non-null int32
ndvi_se                                 1429 non-null int32
ndvi_sw                                 1429 non-null int32
reanalysis_air_temp_k                   1419 non-null float64
reanalysis_max_air_temp_k               1419 non-null float64
reanalysis_min_air_temp_k               1419 non-null float64
reanalysis_relative_humidity_percent    1419 non-null float64
reanalysis_tdtr_k                       1419 non-null float64
station_diur_temp_rng_c                 1388 non-null float64
total_cases                             1429 non-null int64
Jan_Apr                                 1429 non-null int32
May_Aug                                 1429 non-null int32
Sep_Dec                                

In [14]:
#Label-Encoding on city attribute
le = LabelEncoder() 
current_dataset['city']= le.fit_transform(current_dataset['city']) 

In [15]:
#change the data type to category for categorical columns
current_dataset.city = current_dataset["city"].astype("category")
current_dataset.Jan_Apr = current_dataset["Jan_Apr"].astype("category")
current_dataset.May_Aug = current_dataset["May_Aug"].astype("category")
current_dataset.Sep_Dec = current_dataset["Sep_Dec"].astype("category")
current_dataset.ndvi_ne = current_dataset["ndvi_ne"].astype("category")
current_dataset.ndvi_nw = current_dataset["ndvi_nw"].astype("category")
current_dataset.ndvi_se = current_dataset["ndvi_se"].astype("category")
current_dataset.ndvi_sw = current_dataset["ndvi_sw"].astype("category")
current_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1429 entries, 936 to 935
Data columns (total 15 columns):
city                                    1429 non-null category
ndvi_ne                                 1429 non-null category
ndvi_nw                                 1429 non-null category
ndvi_se                                 1429 non-null category
ndvi_sw                                 1429 non-null category
reanalysis_air_temp_k                   1419 non-null float64
reanalysis_max_air_temp_k               1419 non-null float64
reanalysis_min_air_temp_k               1419 non-null float64
reanalysis_relative_humidity_percent    1419 non-null float64
reanalysis_tdtr_k                       1419 non-null float64
station_diur_temp_rng_c                 1388 non-null float64
total_cases                             1429 non-null int64
Jan_Apr                                 1429 non-null category
May_Aug                                 1429 non-null category
Sep_Dec            

In [16]:
#changing the column names
current_dataset.columns = current_dataset.columns.str.replace("station","stn")
current_dataset.columns = current_dataset.columns.str.replace("reanalysis","re_an")
current_dataset.columns = current_dataset.columns.str.replace("humidity","hd")
current_dataset.columns = current_dataset.columns.str.replace("precipitation","prec")
current_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1429 entries, 936 to 935
Data columns (total 15 columns):
city                         1429 non-null category
ndvi_ne                      1429 non-null category
ndvi_nw                      1429 non-null category
ndvi_se                      1429 non-null category
ndvi_sw                      1429 non-null category
re_an_air_temp_k             1419 non-null float64
re_an_max_air_temp_k         1419 non-null float64
re_an_min_air_temp_k         1419 non-null float64
re_an_relative_hd_percent    1419 non-null float64
re_an_tdtr_k                 1419 non-null float64
stn_diur_temp_rng_c          1388 non-null float64
total_cases                  1429 non-null int64
Jan_Apr                      1429 non-null category
May_Aug                      1429 non-null category
Sep_Dec                      1429 non-null category
dtypes: category(8), float64(6), int64(1)
memory usage: 101.2 KB


In [17]:
#create a dataframe for numerical attribute
dengue_num = current_dataset.drop(["city","Jan_Apr","May_Aug","Sep_Dec","ndvi_ne","ndvi_nw","ndvi_se","ndvi_sw","re_an_relative_hd_percent",
                                   "total_cases"], axis =1)

In [18]:
# Standardize the numerical attributes
sd = StandardScaler()
std_dataset = pd.DataFrame(sd.fit_transform(dengue_num))
std_dataset

,0,1,2,3,4
0,-1.429226,1.195519,-1.012560,1.129120,1.265491
1,-1.506709,0.979471,-1.791536,1.519524,1.636138
2,-1.666911,0.331326,-1.207304,0.694444,1.589319
3,-2.442789,0.053550,-2.765257,1.181442,1.152347
4,-1.654346,1.102927,-1.635741,1.290111,1.595562
...,...,...,...,...,...
1424,-1.269025,-1.150148,-0.506225,-0.573363,-1.070416
1425,-0.446029,-1.119284,0.311700,-0.621660,-0.588854
1426,-0.901504,-1.242740,0.039059,-0.770577,-1.103858
1427,-0.776903,-1.150148,0.078007,-0.601536,-0.856388


In [19]:
# assigning the columnn names
std_dataset.columns = dengue_num.columns
std_dataset.head(2)

,re_an_air_temp_k,re_an_max_air_temp_k,re_an_min_air_temp_k,re_an_tdtr_k,stn_diur_temp_rng_c
0,-1.429226,1.195519,-1.012560,1.129120,1.265491
1,-1.506709,0.979471,-1.791536,1.519524,1.636138


In [20]:
re_an_relative_hd_percent = current_dataset["re_an_relative_hd_percent"]
std_dataset["re_an_relative_hd_percent"] = re_an_relative_hd_percent

In [21]:
std_dataset.isnull().sum()

re_an_air_temp_k             10
re_an_max_air_temp_k         10
re_an_min_air_temp_k         10
re_an_tdtr_k                 10
stn_diur_temp_rng_c          41
re_an_relative_hd_percent    36
dtype: int64

In [22]:
std_dataset = std_dataset.fillna(std_dataset.mean())
std_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1429 entries, 0 to 1428
Data columns (total 6 columns):
re_an_air_temp_k             1429 non-null float64
re_an_max_air_temp_k         1429 non-null float64
re_an_min_air_temp_k         1429 non-null float64
re_an_tdtr_k                 1429 non-null float64
stn_diur_temp_rng_c          1429 non-null float64
re_an_relative_hd_percent    1429 non-null float64
dtypes: float64(6)
memory usage: 67.1 KB


In [23]:
# extract categorical and target columns
cat_dataset= current_dataset[['city','Jan_Apr','May_Aug','Sep_Dec','ndvi_sw','ndvi_se','ndvi_nw','ndvi_ne','total_cases']]
#setting the same index for numeraical and categorical dataframe 
cat_dataset.index = std_dataset.index

In [24]:
#concat the numerical, categorical and target attributes after transformations
final_df = pd.concat([std_dataset,cat_dataset],axis=1)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1429 entries, 0 to 1428
Data columns (total 15 columns):
re_an_air_temp_k             1429 non-null float64
re_an_max_air_temp_k         1429 non-null float64
re_an_min_air_temp_k         1429 non-null float64
re_an_tdtr_k                 1429 non-null float64
stn_diur_temp_rng_c          1429 non-null float64
re_an_relative_hd_percent    1429 non-null float64
city                         1429 non-null category
Jan_Apr                      1429 non-null category
May_Aug                      1429 non-null category
Sep_Dec                      1429 non-null category
ndvi_sw                      1429 non-null category
ndvi_se                      1429 non-null category
ndvi_nw                      1429 non-null category
ndvi_ne                      1429 non-null category
total_cases                  1429 non-null int64
dtypes: category(8), float64(6), int64(1)
memory usage: 90.2 KB


#### splitting the data

Implemented the startified splitting to maintain the ratio of records for both the cities.

In [25]:
#performing stratified split on city to maintain the ratio of records for city sj and iq in train and test dataset
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index, test_index in split.split(final_df,final_df["city"]):
    strat_train_set = final_df.loc[train_index]
    strat_test_set = final_df.loc[test_index]

In [26]:
#verifying the ratio 
print(strat_train_set["city"].value_counts()/len(strat_train_set)) 
print(strat_test_set["city"].value_counts()/len(strat_test_set)) 

1    0.641295
0    0.358705
Name: city, dtype: float64
1    0.63986
0    0.36014
Name: city, dtype: float64


In [27]:
# drop the target coulmn and create a target variable dataframe separately
train = strat_train_set.drop("total_cases", axis = 1)
train_target = strat_train_set["total_cases"].copy()
test = strat_test_set.drop("total_cases", axis = 1)
test_target = strat_test_set["total_cases"].copy()
print("train :",train.shape)
print("train_target :",train_target.shape)
print("test :",test.shape)
print("test_target :",test_target.shape)

train : (1143, 14)
train_target : (1143,)
test : (286, 14)
test_target : (286,)


#### Train the Model

3. Optional: Build a Neural Network Multi-Layer Perceptron Regressor model (you can use sklearn neural network MLP Regressor)

In [28]:
mlp_reg = MLPRegressor()
#     hidden_layer_sizes=(200,100,50), activation='relu',max_iter=100, early_stopping=True,
#                        solver='sgd')
mlp_reg.fit(train,train_target)

C:\Users\ankit\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

#### Fine-tuning the model

I have used grid search to find out the best set of hyperparameters for this model.

In [29]:
# from sklearn.model_selection import GridSearchCV
# param_grid_LinearSVR = [
#         {"hidden_layer_sizes" : [(200,),(200,100,50),(200,100),(100,80,50,20) ],
#          "activation" : ['relu','tanh','identity','logistic'],
#        "solver" : ['sgd','adam']
#           }]
# # 
# #"max_iter" :[10,50,100]
# grid1 = GridSearchCV(mlp_reg, param_grid_LinearSVR,scoring = "neg_mean_absolute_error",
#                     return_train_score= True)
# grid1.fit(train,train_target) 

In [30]:
# # summarize the results of the grid search
# print(grid1.best_score_)
# print(grid1.best_params_)

In [31]:
mlp_reg = MLPRegressor( hidden_layer_sizes=(200,), activation='tanh',
                       solver='sgd',random_state=140)
mlp_reg.fit(train,train_target)

C:\Users\ankit\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(200,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=140, shuffle=True, solver='sgd',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [32]:
# predictions on test data
dengue_predictions_mlp_reg =  mlp_reg.predict(test)
mlp_reg_mae = mean_absolute_error(test_target,dengue_predictions_mlp_reg)
print("mean absolute error:",mlp_reg_mae)

mean absolute error: 13.798104404705644


4. Optional: Evaluate the model and compare it with the previous three assignments

On tuning the hyperparameters of the MLP Regressor model, there is slight decrease in MAE.

The best MAE received from 3 models are below on test:-

SGD - 15.85

Linear SVR - 14.799

SVR with Linear Kernel - 14.785

**MLP Regressor - 13.79**

MLP Regressor model worked better than other models used in our analysis.In this model we have 3 layers, input layer, hidden layer of 200 nodes and an output layer. Since, we only have 1143 data points to train using more hidden layers makes the complex and doesn't improve on results.The activation function used is tanh.The range of the tanh function is from (-1 to 1). sgd (stochastic gradient descent) is used for weight optimization.

5. Add a new column called 'above_average' with value 1 or 0. 1 if the total_cases > median of total_cases

In [33]:
final_df['above_average'] = [1 if x >=final_df["total_cases"].median() else 0 for x in final_df['total_cases']] 

In [34]:
print("Median of total cases:",final_df['total_cases'].median())
final_df.head()

Median of total cases: 11.0


,re_an_air_temp_k,re_an_max_air_temp_k,re_an_min_air_temp_k,re_an_tdtr_k,stn_diur_temp_rng_c,re_an_relative_hd_percent,city,Jan_Apr,May_Aug,Sep_Dec,ndvi_sw,ndvi_se,ndvi_nw,ndvi_ne,total_cases,above_average
0,-1.429226,1.195519,-1.012560,1.129120,1.265491,0.733657,0,0,1,0,0,1,0,0,0,0
1,-1.506709,0.979471,-1.791536,1.519524,1.636138,0.773686,0,0,1,0,0,1,0,0,0,0
2,-1.666911,0.331326,-1.207304,0.694444,1.589319,0.820529,0,0,1,0,0,0,0,0,0,0
3,-2.442789,0.053550,-2.765257,1.181442,1.152347,0.803371,0,0,1,0,0,1,0,0,0,0
4,-1.654346,1.102927,-1.635741,1.290111,1.595562,0.804600,0,0,1,0,1,1,1,1,0,0


6. Build a Neural Network MLP Classifier on the 'above_average' column with 80/20 train/test split

In [35]:
#performing stratified split on city to maintain the ratio of records for city sj and iq in train and test dataset
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index, test_index in split.split(final_df,final_df["city"]):
    strat_train_set = final_df.loc[train_index]
    strat_test_set = final_df.loc[test_index]

In [36]:
#verifying the ratio 
print(strat_train_set["city"].value_counts()/len(strat_train_set)) 
print(strat_test_set["city"].value_counts()/len(strat_test_set)) 

1    0.641295
0    0.358705
Name: city, dtype: float64
1    0.63986
0    0.36014
Name: city, dtype: float64


In [37]:
# drop the target coulmn and create a target variable dataframe separately
train = strat_train_set.drop(["total_cases","above_average"], axis = 1)
train_target = strat_train_set["above_average"].copy()
test = strat_test_set.drop(["total_cases","above_average"], axis = 1)
test_target = strat_test_set["above_average"].copy()
print("train :",train.shape)
print("train_target :",train_target.shape)
print("test :",test.shape)
print("test_target :",test_target.shape)

train : (1143, 14)
train_target : (1143,)
test : (286, 14)
test_target : (286,)


Using the set of hyperparameters that resulted in least MAE achieved from hit and trial

In [38]:
mlp_clf = MLPClassifier(hidden_layer_sizes=(200,100,80,),max_iter=1000,activation='relu',solver='sgd',
                        random_state=42)

mlp_clf.fit(train, train_target)

C:\Users\ankit\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200, 100, 80), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=42, shuffle=True, solver='sgd',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [39]:
y_pred = mlp_clf.predict(test)

In [40]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_target, y_pred)

array([[103,  39],
       [ 24, 120]], dtype=int64)

From above confusion matrix we can infer that-

* The model has correctly classified 103 data points for which the total cases were below the median of all total cases and 120 data points for which the total cases were above the median of all the total cases.


* The model misclassified 24 records as total cases above median of total cases when it is below actually and misclassified 39 data points as total cases below median of total cases when it is above actually.

In [41]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_target, y_pred))

0.7797202797202797


In [42]:
from sklearn.metrics import classification_report
print(classification_report(test_target, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.73      0.77       142
           1       0.75      0.83      0.79       144

    accuracy                           0.78       286
   macro avg       0.78      0.78      0.78       286
weighted avg       0.78      0.78      0.78       286



7. Explain the meaning of Precision, Recall and F1-Score and why these are used to evaluate Classification models (instead of using Accuracy as a metric). Evaluate the classifier using Precision, Recall and F1 score values

**Precision-** 

*Precision = TP/TP+FP*

Precision is the ratio of correctly predicted positive observations to the total predicted positive observations.This metric helps to answer that out of all the cases labeled as above average, how many actually is above average.High precision relates to the low false positive rate.

**Recall-** 

*Recall = TP/TP+FN*

Recall is the ratio of correctly predicted positive observations to the all observations in actual true positive. Recall helps to understand of all the total_cases records that were above average how many did we labeled correctly.

**F1-Score-**

*F1 = 2 * (precision * recall) / (precision + recall)*

F1 Score is the weighted average of both Precision and Recall.
These metrics are used to evaluate classification models instead of accuracy because accuracy might include huge contribution of true negatives when there is uneven class distribution in the dataset while these metrics focuses on both false positives and false negatives.



**As per the classification report of our model-**

* The precision achieved for class 0 is better than class 1 and vice versa for recall.


* The precision for our model is 0.78 i.e 78% of the data points were correctly classified.


* The recall for our model is 0.78 i.e. 78% of the data points were correctly predicted of all the observed points actually in a particular class.


* The weighted average of both precision and recall i.e. F1 score is 78%.

## Perform the same analysis on dengue_features_test and create submission file

In [43]:
dengue_features_test = pd.read_csv("dengue_features_test.csv")
print(dengue_features_test.shape)
dengue_features_test.head()

(416, 24)


,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


In [44]:
dengue_features_test_final = dengue_features_test.drop(["precipitation_amt_mm","reanalysis_sat_precip_amt_mm","reanalysis_dew_point_temp_k"
                                                    ,"reanalysis_specific_humidity_g_per_kg","reanalysis_precip_amt_kg_per_m2","reanalysis_avg_temp_k",
                                                    "station_precip_mm","station_max_temp_c","station_avg_temp_c","station_min_temp_c","year","weekofyear"],axis=1)
dengue_features_test_final.head()

,city,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,reanalysis_air_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_relative_humidity_percent,reanalysis_tdtr_k,station_diur_temp_rng_c
0,sj,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,298.492857,301.1,296.4,78.781429,3.128571,7.057143
1,sj,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,298.475714,300.8,296.7,78.230000,2.571429,5.557143
2,sj,2008-05-13,-0.0015,NaN,0.151083,0.091529,299.455714,302.2,296.4,78.270000,4.428571,7.785714
3,sj,2008-05-20,NaN,-0.019867,0.124329,0.125686,299.690000,303.0,296.9,73.015714,4.342857,6.271429
4,sj,2008-05-27,0.0568,0.039833,0.062267,0.075914,299.780000,302.3,297.3,74.084286,3.542857,7.085714


In [45]:
dengue_features_test_final["reanalysis_relative_humidity_percent"] = dengue_features_test_final["reanalysis_relative_humidity_percent"]/100

In [46]:
dengue_features_test_final['ndvi_ne'] = np.where(dengue_features_test_final["ndvi_ne"] <= 0.25, 0,1) 
dengue_features_test_final['ndvi_nw'] = np.where(dengue_features_test_final["ndvi_ne"] <= 0.25, 0,1) 
dengue_features_test_final['ndvi_se'] = np.where(dengue_features_test_final["ndvi_se"] <= 0.25, 0,1) 
dengue_features_test_final['ndvi_sw'] = np.where(dengue_features_test_final["ndvi_sw"] <= 0.25, 0,1) 

In [47]:
#create new columns by extracting the month from week_start_date
dengue_features_test_final["month"] = pd.to_datetime(dengue_features_test_final["week_start_date"]).dt.month
dengue_features_test_final["Jan_Apr"]= np.where(dengue_features_test_final["month"].isin([1,2,3,4]),1,0)
dengue_features_test_final["May_Aug"]= np.where(dengue_features_test_final["month"].isin([5,6,7,8]),1,0)
dengue_features_test_final["Sep_Dec"]= np.where(dengue_features_test_final["month"].isin([9,10,11,12]),1,0)
dengue_features_test_final = dengue_features_test_final.drop(["month","week_start_date"],axis =1)

In [48]:
#Label-Encoding on city attribute
le = LabelEncoder() 
dengue_features_test_final['city']= le.fit_transform(dengue_features_test_final['city']) 

In [49]:
dengue_features_test_final.city = dengue_features_test_final["city"].astype("category")
dengue_features_test_final.Jan_Apr = dengue_features_test_final["Jan_Apr"].astype("category")
dengue_features_test_final.May_Aug = dengue_features_test_final["May_Aug"].astype("category")
dengue_features_test_final.Sep_Dec = dengue_features_test_final["Sep_Dec"].astype("category")
dengue_features_test_final.ndvi_ne = dengue_features_test_final["ndvi_ne"].astype("category")
dengue_features_test_final.ndvi_nw = dengue_features_test_final["ndvi_nw"].astype("category")
dengue_features_test_final.ndvi_se = dengue_features_test_final["ndvi_se"].astype("category")
dengue_features_test_final.ndvi_sw = dengue_features_test_final["ndvi_sw"].astype("category")

In [50]:
dengue_features_test_final.columns = dengue_features_test_final.columns.str.replace("station","stn")
dengue_features_test_final.columns = dengue_features_test_final.columns.str.replace("reanalysis","re_an")
dengue_features_test_final.columns = dengue_features_test_final.columns.str.replace("humidity","hd")
dengue_features_test_final.columns = dengue_features_test_final.columns.str.replace("precipitation","prec")

In [51]:
dengue_features_test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 14 columns):
city                         416 non-null category
ndvi_ne                      416 non-null category
ndvi_nw                      416 non-null category
ndvi_se                      416 non-null category
ndvi_sw                      416 non-null category
re_an_air_temp_k             414 non-null float64
re_an_max_air_temp_k         414 non-null float64
re_an_min_air_temp_k         414 non-null float64
re_an_relative_hd_percent    414 non-null float64
re_an_tdtr_k                 414 non-null float64
stn_diur_temp_rng_c          404 non-null float64
Jan_Apr                      416 non-null category
May_Aug                      416 non-null category
Sep_Dec                      416 non-null category
dtypes: category(8), float64(6)
memory usage: 23.6 KB


In [52]:
#create a dataframe for numerical attribute
dengue_num_test = dengue_features_test_final.drop(["city","Jan_Apr","May_Aug","Sep_Dec","ndvi_ne","ndvi_nw","ndvi_se","ndvi_sw","re_an_relative_hd_percent"], axis =1)

In [53]:
# Standardize the numerical attributes
sd = StandardScaler()
std_dataset_test = pd.DataFrame(sd.fit_transform(dengue_num_test))

In [54]:
std_dataset_test.columns = dengue_num_test.columns

In [55]:
re_an_relative_hd_percent = dengue_features_test_final["re_an_relative_hd_percent"]
std_dataset_test["re_an_relative_hd_percent"] = re_an_relative_hd_percent

In [56]:
std_dataset_test.isnull().sum()

re_an_air_temp_k              2
re_an_max_air_temp_k          2
re_an_min_air_temp_k          2
re_an_tdtr_k                  2
stn_diur_temp_rng_c          12
re_an_relative_hd_percent     2
dtype: int64

In [57]:
std_dataset_test = std_dataset_test.fillna(std_dataset_test.mean())

In [58]:
# extract categorical and target columns
cat_dataset_test= dengue_features_test_final[['city','Jan_Apr','May_Aug','Sep_Dec','ndvi_sw','ndvi_se','ndvi_nw','ndvi_ne']]

In [59]:
cat_dataset_test.index = std_dataset_test.index

In [60]:
#concat the numerical, categorical and target attributes after transformations
final_df_test = pd.concat([std_dataset_test,cat_dataset_test],axis=1)

final_df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 14 columns):
re_an_air_temp_k             416 non-null float64
re_an_max_air_temp_k         416 non-null float64
re_an_min_air_temp_k         416 non-null float64
re_an_tdtr_k                 416 non-null float64
stn_diur_temp_rng_c          416 non-null float64
re_an_relative_hd_percent    416 non-null float64
city                         416 non-null category
Jan_Apr                      416 non-null category
May_Aug                      416 non-null category
Sep_Dec                      416 non-null category
ndvi_sw                      416 non-null category
ndvi_se                      416 non-null category
ndvi_nw                      416 non-null category
ndvi_ne                      416 non-null category
dtypes: category(8), float64(6)
memory usage: 23.6 KB


In [61]:
mlp_reg_test = MLPRegressor( hidden_layer_sizes=(200,), activation='tanh',
                       solver='sgd',random_state=140)
mlp_reg_test.fit(train,train_target)
# predictions on test data
dengue_predictions_mlp_reg_test =  mlp_reg.predict(final_df_test)

In [62]:
#Create a  DataFrame with the city,months divided,and total cases
submission = pd.DataFrame({'city':dengue_features_test['city'],'year': dengue_features_test['year'],'weekofyear':dengue_features_test['weekofyear'],
                           'total_cases':dengue_predictions_mlp_reg_test})
#Visualize the first 5 rows
submission.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,16.712229
1,sj,2008,19,6.195672
2,sj,2008,20,5.367394
3,sj,2008,21,10.831094
4,sj,2008,22,10.836622


#### Evaluate the results achieved

In [63]:
comparison = submission[(submission['year'] == 2012)]
print(comparison)

    city  year  weekofyear  total_cases
191   sj  2012          52    20.148273
192   sj  2012           1    18.973758
193   sj  2012           2    19.750297
194   sj  2012           3    20.374665
195   sj  2012           4    18.180819
..   ...   ...         ...          ...
385   iq  2012          47     6.759028
386   iq  2012          48     6.761670
387   iq  2012          49     6.984058
388   iq  2012          50     8.538365
389   iq  2012          51    10.251537

[104 rows x 4 columns]


We know that the seasons of both the cities varies throughout the year and that has different impact of season on the total cases of dengue in that city. 
On comparing the results for both the cities for 2012 in test data we can conclude that- 

* Like we saw in our training data, the number of total cases of dengue predicted seems to be more in San Juan than Iquitos.


* Similarly, our model predicted that there are more number of dengue cases during initial few weeks of the year and then lesser cases for weeks in middle until 25 to 30th week of the year for both the cities, followed with increase in number of cases in later weeks of several years in the data.

In [66]:
#Convert DataFrame to a csv file that can be uploaded
#This is saved in the same directory as notebook
filename = 'Dengue_Prediction_Submission_final3.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: Dengue_Prediction_Submission_final3.csv


Submitted the file in competition and got a score of 25.95 and current rank 1518.